In [1]:
#### Load Data ####
import numpy as np
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator

import tensorflow as tf
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0" 
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.85
session = tf.Session(config=config)
    
def normalize(X_train,X_test):
    #this function normalize inputs for zero mean and unit variance
    # it is used when training a model.
    # Input: training set and test set
    # Output: normalized training set and test set according to the trianing set statistics.
    mean = np.mean(X_train,axis=(0,1,2,3))
    std = np.std(X_train, axis=(0, 1, 2, 3))
    X_train = (X_train-mean)/(std+1e-7)
    X_test = (X_test-mean)/(std+1e-7)
    return X_train, X_test

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train, x_test = normalize(x_train, x_test)

y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

datagen = ImageDataGenerator(
            featurewise_center=False,  # set input mean to 0 over the dataset
            samplewise_center=False,  # set each sample mean to 0
            featurewise_std_normalization=False,  # divide inputs by std of the dataset
            samplewise_std_normalization=False,  # divide each input by its std
            zca_whitening=False,  # apply ZCA whitening
            rotation_range=15,  # randomly rotate images in the range (degrees, 0 to 180)
            width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
            height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
            horizontal_flip=True,  # randomly flip images
            vertical_flip=False)  # randomly flip images

# (std, mean, and principal components if ZCA whitening is applied).
datagen.fit(x_train)

Using TensorFlow backend.
/LOCAL//anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/LOCAL//anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/LOCAL//anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/LOCAL//anaconda3/lib/python3.7/site-packages/tensorflow/python/framewo

In [2]:
from keras import backend as K

def get_weight_grad(model, inputs, outputs):
    """ Gets gradient of model for given inputs and outputs for all weights"""
    grads = model.optimizer.get_gradients(model.total_loss, model.trainable_weights)
    symb_inputs = (model._feed_inputs + model._feed_targets + model._feed_sample_weights)
    f = K.function(symb_inputs, grads)
    x, y, sample_weight = model._standardize_user_data(inputs, outputs)
    output_grad = f(x + y + sample_weight)
    return output_grad

def get_layer_output_grad(model, inputs, outputs, layer=-1):
    """ Gets gradient a layer output for given inputs and outputs"""
    grads = model.optimizer.get_gradients(model.total_loss, model.layers[layer].output)
    symb_inputs = (model._feed_inputs + model._feed_targets + model._feed_sample_weights)
    f = K.function(symb_inputs, grads)
    x, y, sample_weight = model._standardize_user_data(inputs, outputs)
    output_grad = f(x + y + sample_weight)
    return output_grad

def tran_a(a_1):
    a_1 = np.mean(a_1, axis=0)
    a_1 = a_1.reshape(a_1.shape[0]*a_1.shape[1],-1)
    a_1 = np.mean(a_1,axis=0)
    return a_1
    
def tran_h(h_2):
    h_2 = np.mean(h_2, axis=0)
    h_2 = h_2.reshape(h_2.shape[0]*h_2.shape[1],-1)
    h_2 = np.mean(h_2,axis=0)+10**-13
    return h_2

In [3]:
from keras import optimizers
from keras import backend as K
K.clear_session()

model = keras.models.load_model('vgg16_dropout/final_139.h5',compile=False)
sgd = optimizers.SGD_test(add_ah=False, add_noise=0.0, a_corr=np.zeros([12,512,512]), h_corr=np.zeros([12,512,512]), lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd,metrics=['accuracy'])

trn_loss, trn_acc = model.evaluate(x_train, y_train,verbose=0)
tst_loss, tst_acc = model.evaluate(x_test, y_test,verbose=0)
print(tst_loss-trn_loss)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
0.2996946773934364


In [4]:
#dropout: -13, -12
#normal: -11, -10

from keras import optimizers

def load(fname, a):
    K.clear_session()

    model = keras.models.load_model(fname+'/final_139.h5',compile=False)
    sgd = optimizers.SGD_test(add_ah=False, add_noise=0.0, a_corr=np.zeros([12,512,512]), h_corr=np.zeros([12,512,512]), lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='categorical_crossentropy', optimizer=sgd,metrics=['accuracy'])

    a_1 = []
    h_2 = []
    num = 0
    layer_1 = K.function([model.layers[0].input], [model.layers[a].output])
    for num in [0,20,40,60,80,100,120]:
        a_1.append(tran_a(layer_1([x_train[num:num+20]])[0]))

    for num in [0,20,40,60,80,100,120]:
        h_2.append(tran_h(get_layer_output_grad(model, x_train[num:num+20], y_train[num:num+20], a+1)[0]))
    
    return a_1, h_2

In [6]:
split = 16

for fname in ['vgg16']:
    a_1, h_2 = load(fname, -11)
    deet_a = []
    for i in range(int(len(a_1[0])/split)):
        a = np.kron(a_1[0][i*split:i*split+split], np.array([a_1[0][i*split:i*split+split]]).reshape(-1,1))
        for j in range(1,len(a_1)):
            a += np.kron(a_1[j][i*split:i*split+split], np.array([a_1[j][i*split:i*split+split]]).reshape(-1,1))
        a = np.abs(np.linalg.inv(a))
        a = a/np.max(a)
        for j in range(len(a)):
            a[j][j] = 1
        deet_a.append(np.linalg.det(a))
    deet_a = np.mean(deet_a)

    deet_h = []
    for e in range(20):
        for i in range(int(len(a_1[0])/split)):
            a = np.kron(h_2[0][i*split:i*split+split], np.array([h_2[0][i*split:i*split+split]]).reshape(-1,1))  
            for j in range(1,len(h_2)):
                a += np.kron(h_2[j][i*split:i*split+split], np.array([h_2[j][i*split:i*split+split]]).reshape(-1,1))
            a = a/np.max(a)
            a += np.random.normal(loc=0.0, scale=0.00001, size=(len(a),len(a)))
            a = np.abs(np.linalg.inv(a))
            a = a/np.max(a)
            for j in range(len(a)):
                a[j][j] = 1
            deet_h.append(np.linalg.det(a))
    deet_h = np.mean(deet_h)     
    print(fname)
    print(deet_a*deet_h)


vgg16
0.04755795554573536


In [8]:
#dropout module

split = 16

for fname in ['vgg16_dropout']:
    a_1, h_2 = load(fname, -13)
    deet_a = []
    for i in range(int(len(a_1[0])/split)):
        a = np.zeros((split,split))
        for e in range(50):
            noise = (np.random.uniform(0,1,split)>0.4).astype(float)
            a += np.kron(a_1[0][i*split:i*split+split]*noise, np.array([a_1[0][i*split:i*split+split]*noise]).reshape(-1,1))
            for j in range(1,len(a_1)):
                a += np.kron(a_1[j][i*split:i*split+split]*noise, np.array([a_1[j][i*split:i*split+split]*noise]).reshape(-1,1))    
        a = np.abs(np.linalg.inv(a/50))
        a = a/np.max(a)
        for j in range(len(a)):
            a[j][j] = 1
        deet_a.append(np.linalg.det(a))
    deet_a = np.mean(deet_a)


    deet_h = []
    for e in range(20):
        for i in range(int(len(a_1[0])/split)):
            a = np.kron(h_2[0][i*split:i*split+split], np.array([h_2[0][i*split:i*split+split]]).reshape(-1,1))  
            for j in range(1,len(h_2)):
                a += np.kron(h_2[j][i*split:i*split+split], np.array([h_2[j][i*split:i*split+split]]).reshape(-1,1))
            a = a/np.max(a)
            a += np.random.normal(loc=0.0, scale=0.00001, size=(len(a),len(a)))
            a = np.abs(np.linalg.inv(a))
            a = a/np.max(a)
            for j in range(len(a)):
                a[j][j] = 1
            deet_h.append(np.linalg.det(a))
    deet_h = np.mean(deet_h)     
    print(fname)
    print(deet_a*deet_h)

vgg16_dropout
0.13978243277932703
